In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df1 = pd.read_csv('pf10xIDC_pheno.csv')
df2 = pd.read_csv('pf10xIDC_counts.csv')


feature_names = df2.iloc[:, 0].tolist()

df2 = df2.drop(df2.columns[0], axis=1)
# feature_names
df2



In [ ]:
X = []
for (columnName, columnData) in df2.iteritems():
    X.append(columnData.values.tolist())
Y = df1.bulk
len(Y)

In [ ]:
df = pd.DataFrame(X, columns = feature_names)

In [ ]:
# x = df.values.tolist()
# print(x == X)

In [ ]:
df['bulk'] = Y

In [ ]:
mcc = make_scorer(matthews_corrcoef)

In [ ]:
x = df.drop(['bulk'], axis=1)

In [ ]:
x

In [ ]:
y = df['bulk']
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.2, stratify = y)
X_train
Y_test

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from genetic_selection import GeneticSelectionCV

In [ ]:
estimator = RandomForestClassifier()
# gene = []

In [ ]:

for i in range(500, 800):
    selector = GeneticSelectionCV(estimator,
                                cv = 5,
                                verbose = 1,
                                scoring = mcc,
                                max_features = i,
                                n_population = 5,
                                crossover_proba = 0.5,
                                mutation_proba = 0.2,
                                n_generations = 2,
                                crossover_independent_proba=0.5,
                                mutation_independent_proba=0.05,
                                n_gen_no_change= 20,
                                caching=True,
                                n_jobs=-1)  
    selector = selector.fit(X_train, Y_train)
    gene = X_train.columns[selector.support_]
#     print(len(gene))
#     for j in gene:
#         print(j, end = " ")
    selected_features = X_train[X_train.columns[selector.support_]]
#     print("Features selected from GA")
#     print(selected_features)
    X_test_temp = X_test[X_train.columns[selector.support_]]
    selected_X = selected_features.values.tolist()
    selected_Y = Y_train
    
    
    print("SVM Model")
    model = SVC()
    model.fit(selected_X, selected_Y)
    y_pred = model.predict(X_test_temp)
    arr = f1_score(Y_test, y_pred, average = None)

    print(classification_report(Y_test, y_pred))
    
    print("LR Model")
    clf = LogisticRegression( multi_class= 'ovr').fit(selected_X, selected_Y)
    y_pred_log = clf.predict(X_test_temp)
    print(classification_report(Y_test, y_pred))
        
    print("RF Model")
    RF = RandomForestClassifier()
    RF.fit(selected_X,selected_Y)
    y_pred_RF = RF.predict(X_test_temp)
    print(classification_report(Y_test, y_pred))
    
    
